In [235]:
%reset -f

In [236]:
import numpy as np
import imp
import os
import math

In [237]:
def get_best_error(infile):
    
    index = []
    best_fit = []
    bound = []
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    for line in lines:
        if len(line.split())>4:
            if line.split()[-2] == '+/-':
                index.append(line.split()[0])
                best_fit.append(line.split()[-3])
    best = dict(zip(index,best_fit)) 
    
    index = []
    
    for line in lines:
        if len(line.split())>3:
            if line.split()[-1][-1]==")":
                if len(line.split()[-1].split(','))>1.1:
                    index.append(line.split()[0])
                    bound.append([line.split()[1], line.split()[2]])

    bounds = dict(zip(index,bound))
    
    
    f.close()
    
    return best, bounds

# infile is for the best-fit table and error result both, 
# just copy the result of 'show free','show fit' and 'error' in xspec. 

In [238]:
def get_par_best_error(infile):
    
    index = []
    parameter = []
    best_fit = []
    bound = []
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    for line in lines:
        if len(line.split())>4: 
            if line.split()[-2] == '+/-':
                if (line.split()[3] == 'norm' or line.split()[3] == 'logxi'):
                    index.append(line.split()[0])
                    parameter.append(line.split()[3] + "_" + line.split()[2])
                else:
                    index.append(line.split()[0])
                    parameter.append(line.split()[3])
    parameters = dict(zip(index,parameter))
    
    
    for line in lines:
        if len(line.split())>4: 
            if line.split()[-2] == '+/-':
                index.append(line.split()[0])
                best_fit.append(line.split()[-3])
    best = dict(zip(index,best_fit)) 
    
    
    for line in lines:
        if len(line.split())>3:
            if line.split()[-1][-1]==")":
                if len(line.split()[-1].split(','))>1.1:
                    index.append(line.split()[0])
                    bound.append([line.split()[1], line.split()[2]])
                    
    bounds = dict(zip(index,bound))
    
    
    f.close()
    
    return parameters, best, bounds

In [239]:
def get_chi_sq_and_AICc_single_file(infile):
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    index = ['chisq','dof','reduced_chisq','N-bin','N-par','AICc']
    value = [None]*6
    
    for line in lines:
        if line.split()[0] == 'Test' and line.split()[-1] == 'bins.':
            value[3] = int(line.split()[-2])
    
    for line in lines:
        if len(line.split())>4:
            if line.split()[-1] == 'd.o.f.':
                value[0] = float(line.split()[3])
                value[1] = int(line.split()[-2])
                value[2] = float("{:.5f}".format(value[0] / value[1]))

    value[4] = value[3] - value[1]
    
    value[5] = float("{:.2f}".format(value[0] + 2*value[4] + 2*value[4]*(value[4]+1)/(value[3]-value[4]-1)))
    
    result = dict(zip(index,value))
    
    return result

In [240]:
def get_latex_elements_single_file(infile):
    import imp
    import error2latex as el
    imp.reload(el)
    
    parameter, best_fit, bounds = get_par_best_error(infile)
    
    #index = []
    par = []
    latex_c = []
    
    if len(best_fit)!=len(bounds):
        print("Error: best-fit parameters do not match the number of uncertainties")
        print(f"Number of free parameters: {len(best_fit)}, number of boundaries: {len(bounds)}")
    
    for key in best_fit.keys():
        name = parameter[key]
        best = best_fit[key]
        lower = bounds[key][0]
        upper = bounds[key][1]
        string, value_base, err_low_base, err_high_base = el.get_xspec_error(best, lower, upper) 
        
        #index.append(key)
        par.append(name)
        latex_c.append(string)    
        
       # print(string)
    #result = dict(zip(index,latex_c))
    result = dict(zip(par,latex_c))
    
    return result

In [241]:
def get_latex_elements_multiple_file(infiles):
    number = len(infiles)
    eles = [None]*number
    for i in range(number):
        eles[i] = get_latex_elements_single_file(infiles[i])
    pars = []
    for i in range(number):
        pars = pars + list(eles[i].keys())
    keys_origin = list(set(pars))
    keys = sort_parameters(keys_origin)
    
    return eles, keys

In [242]:
def sort_parameters(key_origin):
    parameter_lib = ['nH',
                     'norm_nthComp',
                     'Gamma','kT_e',
                     'Index1','Index2','Index3','Rbr1','Rbr2','a','Incl','logxi_relxillCp','Afe','norm_relxillCp',
                     'logxi_xillverCp','norm_xillverCp',
                     'factor'] #在此函数中设定表格中参数的顺序，根据需求修改
    a = range(len(parameter_lib))
    dict1 = dict(zip(a,parameter_lib))
    dict2 = dict(zip(parameter_lib,a))
    key_temp_1 = []
    keys = []
    
    for i in key_origin:
        key_temp_1.append(dict2.get(i))
        
    key_temp_2 = sorted(key_temp_1)  
    
    for j in key_temp_2:
        keys.append(dict1.get(j))
        
    return keys

In [243]:
def print_chi_sq_and_AICc_multiple_files(infiles):
    
    number = len(infiles)
    chisq, dof, reduced_chisq, AICc = [None]*number, [None]*number, [None]*number, [None]*number
    for i in range(number):
        chisq[i] = get_chi_sq_and_AICc_single_file(infiles[i]).get('chisq')
        dof[i] = get_chi_sq_and_AICc_single_file(infiles[i]).get('dof')
        reduced_chisq[i] = get_chi_sq_and_AICc_single_file(infiles[i]).get('reduced_chisq')
        AICc[i] = get_chi_sq_and_AICc_single_file(infiles[i]).get('AICc')

    string = '$\chi^2$ /dof'
    
    string_AICc = 'AICc'
    
    for n in range(number):
        string = string + ' & ' +'\\tabincell{c}{' + str(chisq[n]) + '/' + str(dof[n]) + '=\\\\' + str(reduced_chisq[n]) + '}'
        string_AICc = string_AICc + ' & ' + str(AICc[n])
        
    print('\\hline' + '\n\n' + string + '\\\\' + '\n\n' + string_AICc +  '\\\\' + '\n\n' + '\\hline' + '\n')

In [244]:
#创建一个字典，键是序列keys里的元素，值是对应的latex字符串
#需根据实际情况进行扩充或修改。
def get_latex_of_parameters(keys):
    number = len(keys)
    keys_latex = [None]*number
    for i in range(number):
        if keys[i] == 'nH':
            keys_latex[i] = '$N_{\\rm H}$ [\\(10^{22}\\) cm\\(^{-2}\\)]'
        if keys[i] == 'norm_nthComp':
            keys_latex[i] = 'Norm({\\tt nthComp})'
        if keys[i] == 'Gamma':
            keys_latex[i] = '$\\Gamma$'
        if keys[i] == 'kT_e':
            keys_latex[i] = '$kT_{\\rm e}$ [keV]'
        if keys[i] == 'Index1':
            keys_latex[i] = '$q_{\\rm in}$'
        if keys[i] == 'Index2':
            keys_latex[i] = '$q_{\\rm out}$ '
        if keys[i] == 'Index3':
            keys_latex[i] = 'Index3'
        if keys[i] == 'Rbr1':
            keys_latex[i] = '$R_{\\rm br1}$ [$R_{\\rm g}$]'
        if keys[i] == 'Rbr2':
            keys_latex[i] = '$R_{\\rm br2}$ [$R_{\\rm g}$]'
        if keys[i] == 'a':
            keys_latex[i] = '$a_{*}$'
        if keys[i] == 'Incl':
            keys_latex[i] = '$i$ [deg]'
        if keys[i] == 'logxi_relxillCp':
            keys_latex[i] = 'log${\\xi}$ [erg~cm~s$^{-1}$]'
        if keys[i] == 'logxi_xillverCp':
            keys_latex[i] = 'log${\\xi}$ [erg~cm~s$^{-1}$]'   
        if keys[i] == 'Afe':
            keys_latex[i] = 'A$_{\\rm Fe}$'
        if keys[i] == 'norm_relxillCp':
            keys_latex[i] = 'Norm({\\tt relxillCp})'
        if keys[i] == 'norm_xillverCp':
            keys_latex[i] = 'Norm({\\tt xillverCp})'
        if keys[i] == 'factor':
            keys_latex[i] = '$C_{\\rm HXD}$ '
    par_latex = dict(zip(keys,keys_latex))
    return par_latex

In [245]:
def updateFile_delete_hashtag_and_blanck_line(file):
    
    file_data = ""
    with open(file,"r", encoding="utf-8") as f:
            lines = f.readlines()
            for line in lines:
                if '#' in line:
                    line = line.replace('#','')
                if line == '\n':
                    line = line.strip("\n")           
                file_data += line
    
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)         

In [246]:
def updateFiles_delete_hashtag_and_blanck_line(infiles):
    for i in range(len(infiles)):
        updateFile_delete_hashtag_and_blanck_line(infiles[i]) 

In [247]:
# 请先保证文档没有空行。
def updateFile_delete_str(file):
    
    file_data = ""
    
    str_delete = ['Apparent','Current','and','but','Suggest','Error','***','caused','Parameter','***Warning:']
    
    with open(file, "r") as input:
        lines = input.readlines()
        for line in lines:
            if line.split() == []:
                continue
            if line.split()[0] not in str_delete:
                file_data += line
    
    with open(file,"w",encoding="utf-8") as output:
        output.write(file_data)    

In [248]:
def updateFiles_delete_str(files):
    for i in range(len(infiles)):
        updateFile_delete_str(infiles[i])

In [249]:
def updateFile_NK(file):
    # 替换文件中的字符串,把rexill_nk的后缀nk去掉。这个函数对需要把relxill和relxill_nk放在同一行比较时才有用。
    file_data = ""
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if 'relxillCp_nk' in line:
                line = line.replace('relxillCp_nk','relxillCp ')
            file_data += line
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)

In [250]:
def updateFile_Rbr(file):
    # 替换文件中的字符串,把Rbr替换为Rbr_1。这个函数对需要把relxill和relxill_nk放在同一行比较时才有用。
    file_data = ""
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if ('Rbr' in line) and ('Rbr1' not in line) and ('Rbr2' not in line):
                line = line.replace('Rbr','Rbr1')
            file_data += line
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)

In [251]:
def updateFiles_NK(infiles):
    for i in range(len(infiles)):
        updateFile_NK(infiles[i]) 

In [252]:
def updateFiles_Rbr(infiles):
    for i in range(len(infiles)):
        updateFile_Rbr(infiles[i]) 

In [253]:
def print_final_latex_code(infiles):
    
    updateFiles_delete_hashtag_and_blanck_line(infiles) #删除所有空行和’#‘
    updateFiles_delete_str(infiles) #删除不需要的文字行
    updateFiles_NK(infiles) #文件预处理，有时候并不需要，请注释掉。
    updateFiles_Rbr(infiles) #文件预处理，有时候并不需要，请注释掉。
    
    eles, keys = get_latex_elements_multiple_file(infiles)
    
    par_latex = get_latex_of_parameters(keys)
    
    print('\\newcommand{\\tabincell}[2]{\\begin{tabular}{@{}#1@{}}#2\\end{tabular}}' + '\n'
          + '%please copy this line to the front of your .tex document in order to use \\tabincell'+ '\n\n' )
    
    print('\\begin{table*}' + '\n' + '\\centering' + '\n' + '\\renewcommand\\arraystretch{1.5}' + '\n'
          + '\\caption{}' + '\n' + '\\label{bestfit}' + '\n' + '\\resizebox{\\textwidth}{70mm}{' )

    chart = '' 
    for i in range(len(infiles)):
        chart += 'c'

    print('\\begin{tabular}{l' + chart + '}'+ '\n')
    
    print('\\hline' + '\n')
    
    model = 'Model'
    for i in range(len(infiles)):
        model += (' & ' + 'Model ' + str(i+1))
    
    print(model + '\\\\' + '\n')
    
    print('\\hline' + '\n')
    
    for par in keys:
        string = par_latex.get(par)
        for n in range(len(infiles)):
            if par in set(list(eles[n].keys())):
                string = string + ' & '+ eles[n][par]
            else:
                string = string + ' & ' + ' - '
        print(string + '\\\\' + '\n')

    print_chi_sq_and_AICc_multiple_files(infiles)
    
    #print AICc
    
    
    print('\\end{tabular}' + '\n' + '}' + '\n' + '\\end{table*}')

In [254]:
infiles = ['relcp_2294.txt', 'relcp_2294 copy.txt']

#在执行前，请根据实际情况修改函数：sort_parameters()，get_latex_of_parameters()，print_final_latex_code()
print_final_latex_code(infiles)

\newcommand{\tabincell}[2]{\begin{tabular}{@{}#1@{}}#2\end{tabular}}
%please copy this line to the front of your .tex document in order to use \tabincell


\begin{table*}
\centering
\renewcommand\arraystretch{1.5}
\caption{}
\label{bestfit}
\resizebox{\textwidth}{70mm}{
\begin{tabular}{lcc}

\hline

Model & Model 1 & Model 2\\

\hline

$N_{\rm H}$ [\(10^{22}\) cm\(^{-2}\)] & $9.11_{-0.07}^{+0.04}$ & $9.11_{-0.07}^{+0.04}$\\

Norm({\tt nthComp}) & $4.67_{-0.16}^{+0.14}$ & $4.67_{-0.16}^{+0.14}$\\

$\Gamma$ & $2.444_{-0.007}^{+0.04}$ & $2.444_{-0.007}^{+0.04}$\\

$kT_{\rm e}$ [keV] & $187_{-44}^{+55}$ & $187_{-44}^{+55}$\\

$q_{\rm in}$ & $10.0_{-1.4}^{+P}$ & $10.0_{-1.4}^{+P}$\\

$q_{\rm out}$  & $0.001_{-P}^{+0.23}$ & $0.001_{-P}^{+0.23}$\\

$R_{\rm br1}$ [$R_{\rm g}$] & $7.37_{-0.22}^{+0.14}$ & $7.37_{-0.22}^{+0.14}$\\

$a_{*}$ & $0.9931_{-0.0019}^{+0.0009}$ & $0.9931_{-0.0019}^{+0.0009}$\\

$i$ [deg] & $73.9_{-0.4}^{+0.3}$ & $73.9_{-0.4}^{+0.3}$\\

log${\xi}$ [erg~cm~s$^{-1}$] & $2.7